In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
# Load the Training data set 
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')

# Display the first 5 rows in the Training Dataset 
train_data.head()

# Display the last 5 rows in the Training dataset
#train_data.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# Load the Test dataset
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')

# Display the first 5 rows of the Test Dataset 
test_data.head()

# Display the last 5 rows of the test dataset 
test_data.tail()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
417,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


In [4]:
# Checking for the women who survived
women = train_data.loc[train_data.Sex == 'female']['Survived']

# Calculate the mean
rate_women = sum(women) / len(women)

# Calculate mean of the women who did't survive
#rate_women_not_survived = 1 - rate_women

# print the output
print('% of the women who survived is : ', rate_women)
#print('% of the women who did not survive is : ', rate_women_not_survived)

% of the women who survived is :  0.7420382165605095


In [5]:
# Checking for men who survived
men  = train_data.loc[train_data.Sex == 'male']['Survived']

# Calculate the mean who survived
rate_men = sum(men) / len(men)

# Calculate the mean of men who didn't survive
#rate_men_not_survived = 1 - rate_men

# print the output
print('% of men who survived is: ', rate_men)
#print('% of men who did not survive is: ', rate_men_not_survived)

% of men who survived is:  0.18890814558058924


In [6]:
# Doing Model selection do determine the best model

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report
from sklearn.metrics import confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Select feature to used 
features = ['Sex', 'SibSp', 'Parch', 'Pclass']

# Convert the text into numerics
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])
y = train_data['Survived']

# Split the train set and test set 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)



# Instatiate the classifiers
classifier = {
    'Logistic Regression' : LogisticRegression(),
    'Decision Tree' : DecisionTreeClassifier(),
    'Random Forest' : RandomForestClassifier(),
    "Support Vector Machine" : SVC(),
    "Nearest Neighbors" : KNeighborsClassifier()
}

# Train and Evaluate the model perfomance
conf_matrices = {} # Store confusion matrices for each classifier
results = {} # Store evaluation metrics for each classifier
class_report = {} # Store the classification report for each classifier

# Iterate through all the classifier
for name, clf in classifier.items():
    print('Fitting models '.format(name))
    clf.fit(X_train, y_train)
    
    # Predicting the models on the test dataset 
    print('....Predicting the model\n')
    y_pred = clf.predict(X_test)
    
    # Calculate the confusion matrix
    con_matrix = confusion_matrix(y_test, y_pred)
    conf_matrices[name] = con_matrix
    
    # Calculate the classification report 
    clas_report = classification_report(y_test, y_pred)
    class_report[name] = clas_report
    

    
# Display the confusion matrix of all the models
for name, con_matrix in conf_matrices.items():
    print(f'The confusion matrix for the model are:\n {name}')
    print(con_matrix)
    print()
    
# Display the classification report of all the models
for name, clas_report in class_report.items():
    print(f"The classification report of the model are: \n{name}")
    print(clas_report)
    print()

# Cross-validation on the clf model
from sklearn.model_selection import cross_val_score

cv_cross = {}

# To perform cross-validation on all the classifiers
for name, clf in classifier.items():
    cv_cross[name] = cross_val_score(clf, X_train, y_train, cv = 5).mean()

# Display the values 
for name, scores in cv_cross.items():
    print(f"The cross validation score of each of the model is as follows: \n{name}")
    print(scores)
    print()

Fitting models 
....Predicting the model

Fitting models 
....Predicting the model

Fitting models 
....Predicting the model

Fitting models 
....Predicting the model

Fitting models 
....Predicting the model

The confusion matrix for the model are:
 Logistic Regression
[[90 15]
 [22 52]]

The confusion matrix for the model are:
 Decision Tree
[[92 13]
 [26 48]]

The confusion matrix for the model are:
 Random Forest
[[92 13]
 [26 48]]

The confusion matrix for the model are:
 Support Vector Machine
[[92 13]
 [24 50]]

The confusion matrix for the model are:
 Nearest Neighbors
[[95 10]
 [35 39]]

The classification report of the model are: 
Logistic Regression
              precision    recall  f1-score   support

           0       0.80      0.86      0.83       105
           1       0.78      0.70      0.74        74

    accuracy                           0.79       179
   macro avg       0.79      0.78      0.78       179
weighted avg       0.79      0.79      0.79       179


The

In [8]:
# Do hyperparameter tuning on the Logisitic Regression
from sklearn.model_selection import GridSearchCV
param_grid = {
    'C' : [0.01, 0.25, 1.0],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': [1000]
}

# Initialize grid search
grid_search = GridSearchCV(LogisticRegression(),param_grid, scoring = 'f1')

# Fit the grid search
grid_search.fit(X_train, y_train)

# The best performed model parameter and score
best_parameter = grid_search.best_params_
print(f"The best parameter is: {best_parameter}")

best_score = grid_search.best_score_
print(f"The best score is: {best_score}")

# Predict the model on the test data
y_pred = grid_search.predict(X_test)

# Creating a column to store the values
results = pd.DataFrame({'PassengerId' : test_data.PassengerId, 'Survived' : y_pred})

# Saving the file as Submission.csv
results.to_csv('Submission.csv', index= False)

# Display output 
print('The file has been successifully submitted!!')

The best parameter is: {'C': 1.0, 'max_iter': 1000, 'solver': 'newton-cg'}
The best score is: 0.7131810376807345


ValueError: array length 179 does not match index length 418